# RDF und SPARQL Sandkasten

In [ ]:
# nötige Module importieren
%pip install -q rdflib
import rdflib
import pandas as pd

In [ ]:
# Funktionsdefinition, um eine Query gegen eine lokale Turtle Datei auszuführen
def query(ttl_string, query_string):
    
    g = rdflib.Graph()
    g.parse(data = ttl_string)

    qres = g.query(query_string)

    df = pd.DataFrame(qres, columns=qres.vars)
    display(df)

## Spezifikationen

Diese technischen Dokumente sind sehr lesenwert (und für Spezifikationen auch sehr leserlich geschrieben):

- [RDF Primer](https://www.w3.org/TR/rdf11-primer/)
- [Turtle](https://www.w3.org/TR/turtle/)
- [SPARQL](https://www.w3.org/TR/sparql11-query/)

## Vanilla TTL

Schreibe ein paar Triples in der Turtle Serialisierung rund um den Politikbetrieb in der Schweiz. Nutze für die URIs `<https://example.com/...>`

In [ ]:
ttl_string = """

<https://example.com/KarinKeller-Sutter> <https://example.com/position> <https://example.com/Bundesrat>.
<https://example.com/KarinKeller-Sutter> <https://example.com/partei> <https://example.com/FDP>.

<https://example.com/ViolaAmherd> <https://example.com/position> <https://example.com/Bundesrat>.
<https://example.com/ViolaAmherd> <https://example.com/partei> <https://example.com/DieMitte>.

<https://example.com/NadineMasshardt> <https://example.com/position> <https://example.com/Nationalrat>.
<https://example.com/NadineMasshardt> <https://example.com/partei> <https://example.com/SP>.
<https://example.com/NadineMasshardt> <https://example.com/geborenIn> <https://example.com/AffolternAmAlbis>.

<https://example.com/MayaGraf> <https://example.com/position> <https://example.com/Staenderat>.
<https://example.com/MayaGraf> <https://example.com/partei> <https://example.com/Gruene>.
<https://example.com/MayaGraf> <https://example.com/vater> <https://example.com/FritzGraf>.

<https://example.com/FritzGraf> <https://example.com/position> <https://example.com/BaselbieterLandrat>.
<https://example.com/FritzGraf> <https://example.com/partei> <https://example.com/SVP>.
<https://example.com/FritzGraf> <https://example.com/kind> <https://example.com/MayaGraf>.

"""

g = rdflib.Graph()
g.parse(data = ttl_string)

In [ ]:
query_string = """


SELECT * WHERE {

    ?br <https://example.com/position> <https://example.com/Bundesrat>.

}

"""

query(ttl_string, query_string)

## TTL mit Prefix

Ersetze in Deinen Triples `<https://example.com/>` durch einen Prefix (am besten den Empty Prefix; [Base vs. Empty Prefix](https://stackoverflow.com/questions/34146707/turtle-difference-between-base-and-empty-prefix))

In [ ]:
ttl_string = """

@prefix : <https://example.com/>.

:KarinKeller-Sutter :position :Bundesrat.
:KarinKeller-Sutter :partei :FDP.

:ViolaAmherd :position :Bundesrat.
:ViolaAmherd :partei :DieMitte.

:NadineMasshardt :position :Nationalrat.
:NadineMasshardt :partei :SP.
:NadineMasshardt :geborenIn :AffolternAmAlbis.

:MayaGraf :position :Staenderat.
:MayaGraf :partei :Gruene.
:MayaGraf :vater :FritzGraf.

:FritzGraf :position :BaselbieterLandrat.
:FritzGraf :partei :SVP.
:FritzGraf :kind :MayaGraf.

"""

In [ ]:
query_string = """

PREFIX : <https://example.com/>


SELECT ?vater ?kind WHERE {

    ?vater :kind ?kind.

}

"""

query(ttl_string, query_string)

## TTL ohne Wiederholungen

Ersetze doppelte Subjekte mit Hilfe von [Prädikat-Listen](https://www.w3.org/TR/turtle/#predicate-lists).

In [ ]:
ttl_string = """

@prefix : <https://example.com/>.

:KarinKeller-Sutter :position :Bundesrat;
    :partei :FDP.

:ViolaAmherd :position :Bundesrat;
    :partei :DieMitte.

:NadineMasshardt :position :Nationalrat;
    :partei :SP;
    :geborenIn :AffolternAmAlbis.

:MayaGraf :position :Staenderat;
    :partei :Gruene;
    :vater :FritzGraf.

:FritzGraf :position :BaselbieterLandrat;
    :partei :SVP;
    :kind :MayaGraf.

"""

In [ ]:
query_string = """

PREFIX : <https://example.com/>


SELECT ?pos (COUNT(?person) as ?count) WHERE {

    ?person :position ?pos.

} GROUP BY ?pos

"""

query(ttl_string, query_string)

## TTL mit echter URI

In [ ]:
ttl_string = """

@prefix : <https://ld.di.digisus-lab.ch/>.

:KarinKeller-Sutter :position :Bundesrat;
    :partei :FDP.

:ViolaAmherd :position :Bundesrat;
    :partei :DieMitte.

:NadineMasshardt :position :Nationalrat;
    :partei :SP;
    :geborenIn :AffolternAmAlbis.

:MayaGraf :position :Staenderat;
    :partei :Gruene;
    :vater :FritzGraf.

:FritzGraf :position :BaselbieterLandrat;
    :partei :SVP;
    :kind :MayaGraf.

"""

In [ ]:
query_string = """

PREFIX : <https://ld.di.digisus-lab.ch/>


SELECT ?partei WHERE {

    ?s :partei ?partei.

}

"""

query(ttl_string, query_string)

## TTL mit externem Vokabular


In [ ]:
ttl_string = """

@prefix : <https://ld.di.digisus-lab.ch/>.
@prefix schema: <http://schema.org/>.

:KarinKeller-Sutter schema:hasOccupation :Bundesrat;
    :partei :FDP.

:ViolaAmherd schema:hasOccupation :Bundesrat;
    :partei :DieMitte.

:NadineMasshardt schema:hasOccupation :Nationalrat;
    :partei :SP;
    :geborenIn :AffolternAmAlbis.

:MayaGraf schema:hasOccupation :Staenderat;
    :partei :Gruene;
    schema:parent :FritzGraf.

:FritzGraf schema:hasOccupation :BaselbieterLandrat;
    :partei :SVP;
    schema:children :MayaGraf.

"""

## TTL mit Literals

In [ ]:
ttl_string = """

@prefix : <https://ld.di.digisus-lab.ch/>.
@prefix schema: <http://schema.org/>.
@prefix xsd: <http://www.w3.org/2001/XMLSchema#>.

:KarinKeller-Sutter schema:hasOccupation :Bundesrat;
    :partei :FDP;
    :anzahlGeschwister 3.

:ViolaAmherd schema:hasOccupation :Bundesrat;
    :partei :DieMitte;
    :vollerName "Viola Patricia Amherd".

:NadineMasshardt schema:hasOccupation :Nationalrat;
    :partei :SP;
    :geborenIn :AffolternAmAlbis;
    schema:birthDate "1984-10-04"^^xsd:date.

:MayaGraf schema:hasOccupation :Staenderat;
    :partei :Gruene;
    schema:parent :FritzGraf.

:FritzGraf schema:hasOccupation :BaselbieterLandrat;
    :partei :SVP;
    schema:children :MayaGraf.

"""

In [ ]:
query_string = """

PREFIX : <https://ld.di.digisus-lab.ch/>
PREFIX schema: <http://schema.org/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>


SELECT ?o WHERE {

    ?s ?p ?o.
    
    FILTER(ISLITERAL(?o)).

}

"""

query(ttl_string, query_string)

## TTL mit Blank Nodes

In [ ]:
ttl_string = """

@prefix : <https://ld.di.digisus-lab.ch/>.
@prefix schema: <http://schema.org/>.
@prefix xsd: <http://www.w3.org/2001/XMLSchema#>.


:Zuerich :population :zh_pop.
:zh_pop :number 423193;
    :valid "2021-12-31"^^xsd:date.

:Bern :population [
    :number 134290;
    :valid "2021-12-31"^^xsd:date ].

"""

In [ ]:
query_string = """

PREFIX : <https://ld.di.digisus-lab.ch/>
PREFIX schema: <http://schema.org/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>


SELECT * WHERE {

    ?city :population ?pop.
    ?pop :number ?number.

}

"""

query(ttl_string, query_string)

In [ ]:
query_string = """

PREFIX : <https://ld.di.digisus-lab.ch/>
PREFIX schema: <http://schema.org/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>


SELECT * WHERE {

    ?city :population/:number ?number.

}

"""

query(ttl_string, query_string)

## OWL

In [ ]:
ttl_string = """

@prefix : <https://ld.di.digisus-lab.ch/>.
@prefix schema: <http://schema.org/>.
@prefix xsd: <http://www.w3.org/2001/XMLSchema#>.
@prefix owl: <http://www.w3.org/2002/07/owl#>.

:KarinKeller-Sutter a :Bundesrat;
    :partei :FDP;
    :anzahlGeschwister 3.

:ViolaAmherd a :Bundesrat;
    :partei :DieMitte;
    :vollerName "Viola Patricia Amherd".

:NadineMasshardt a :Nationalrat;
    :partei :SP;
    :geborenIn :AffolternAmAlbis;
    schema:birthDate "1984-10-04"^^xsd:date.

:MayaGraf a :Staenderat;
    :partei :Gruene;
    schema:parent :FritzGraf.

:FritzGraf a :BaselbieterLandrat;
    :partei :SVP;
    schema:children :MayaGraf.
    
    
:partei owl:inverseOf :parteimitglied.

:Bundesratspartei owl:equivalentClass [
    a owl:Restriction;
    owl:onProperty :parteimitglied;
    owl:someValuesFrom :Bundesrat ].

"""

In [ ]:
query_string = """

PREFIX : <https://ld.di.digisus-lab.ch/>
PREFIX schema: <http://schema.org/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>


SELECT * WHERE {

    ?partei a :Bundesratspartei.

}

"""

query(ttl_string, query_string)

Für OWL Ableitungen braucht es einen "echten" Triple Store, siehe bspw. https://ld.di.digisus-lab.ch/DieMitte

## SPARQL Base Query

In [ ]:
query_string = """

PREFIX : <https://ld.di.digisus-lab.ch/>
PREFIX schema: <http://schema.org/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>


SELECT * WHERE {

    ?s ?p ?o.

}

"""

query(ttl_string, query_string)